<a href="https://colab.research.google.com/github/MichaelPaulukonis/notebooks/blob/main/pharmapsychotic_ImageSorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ImageSorter

This notebook lets you sort a folder of images by perceptual similarity. Why would you want to do this? 
* Sort your StyleGAN seed images for smoother interpolation animations
* Sort generative image variations then use FiLM for cool animations
* Sort generative image variations then make into animated GIF
* Sort images for a collection and look for dupes or ones too similar
* Sort art work in a pleasing progression for a metaverse show
* Sort things for fun because you're bored 

<br>

Limitations:
* If you have a ton of images you may run out of RAM or finding the optimal order may take a long time.

<br>

If you're looking for more Ai art tools check out my [Ai generative art tools list](https://pharmapsychotic.com/tools.html).


In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-3cac0ed3-9b85-8885-04a1-df69568c066f)


In [2]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#@title Setup
!pip install lpips -q
!pip install python-tsp -q

import lpips
import numpy as np
import os
import PIL.Image
import shutil
import torch
import torchvision.transforms.functional as TF
from tqdm import tqdm
from python_tsp.heuristics import solve_tsp_simulated_annealing

def load_image(path, dims):
    image = PIL.Image.open(path).convert("RGB")
    image = image.resize((dims, dims), PIL.Image.BILINEAR)
    return TF.to_tensor(image).to(device).mul(2).sub(1)

     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 


In [8]:
#@title Let's sort some images!
images_folder = "/content/gdrive/MyDrive/Art general/FAKE_POPART_ANATOMY" #@param {type:"string"}
sorted_folder = "/content/gdrive/MyDrive/Art general/anatomy_sorted" #@param {type:"string"}
perceptual_model = 'alex' #@param ['alex', 'squeeze', 'vgg']  
max_sort_minutes = 5 #@param {type: "integer"}
max_images = 300 #@param {type: "integer"}

device = torch.device(f'cuda:0')
lpips_model = lpips.LPIPS(net=perceptual_model).to(device)

files = [file for file in os.listdir(images_folder) if os.path.splitext(file)[1] in ['.png', '.jpg', '.jpeg']]
if not len(files):
    raise Exception(f"No image files found in {images_folder}")

gpu_images = []
for file in tqdm(files, desc="Loading images"):
    gpu_images.append(load_image(os.path.join(images_folder, file), 128))

image_count = min(max_images, len(gpu_images)) if max_images != -1 else len(gpu_images)

indexes = [i for i in range(image_count)]
distances = np.zeros((len(indexes), len(indexes)))
for a in tqdm(range(len(indexes)), desc="Computing distances"):
    for b in range(a+1, len(indexes)):
        diff = lpips_model(gpu_images[a], gpu_images[b])
        distances[a][b] = diff
        distances[b][a] = diff

print(f"Solving for optimal order using up to {max_sort_minutes} minutes...")
ordering, total_distance = solve_tsp_simulated_annealing(distances, max_processing_time=max_sort_minutes*60)

print(f"Saving in sorted order to {sorted_folder}...")
if not os.path.exists(sorted_folder):
    os.makedirs(sorted_folder)
for i in range(image_count):
    file = files[ordering[i]] 
    base, ext = os.path.splitext(file)
    dest = os.path.join(sorted_folder, f"{i:04d}_{base}.{ext}")
    if os.path.exists(dest):
        os.remove(dest)
    shutil.copyfile(os.path.join(images_folder, file), dest)

print("Sorted order:")
print([os.path.splitext(files[ordering[i]])[0] for i in range(len(ordering))])


Saving in sorted order to /content/gdrive/MyDrive/Art general/anatomy_sorted...
Sorted order:
['tile.00.1145', 'tile.00.1221', 'tile.00.264', 'tile.00.266', 'tile.00.261', 'tile.00.263', 'tile.00.061', 'tile.00.063', 'tile.00.064', 'tile.00.272', 'tile.00.303', 'tile.00.306', 'tile.00.844', 'tile.00.288', 'tile.00.301', 'tile.00.299', 'tile.00.317', 'tile.00.1157', 'tile.00.1153', 'tile.00.1168', 'tile.00.869', 'tile.00.881', 'tile.00.930', 'tile.00.926', 'tile.00.1178', 'tile.00.1019', 'tile.00.1026', 'tile.00.1196', 'tile.00.738', 'tile.00.729', 'tile.00.739', 'tile.00.688', 'tile.00.674', 'tile.00.672', 'tile.00.676', 'tile.00.1169', 'tile.00.854', 'tile.00.895', 'tile.00.853', 'tile.00.897', 'tile.00.715', 'tile.00.846', 'tile.00.883', 'tile.00.879', 'tile.00.262', 'tile.00.671', 'tile.00.677', 'tile.00.705', 'tile.00.701', 'tile.00.667', 'tile.00.1143', 'tile.00.1231', 'tile.00.1208', 'tile.00.1021', 'tile.00.919', 'tile.00.505', 'tile.00.528', 'tile.00.1008', 'tile.00.1024', 'til